In [1]:
#r "nuget: Microsoft.SemanticKernel, *-*"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, *-*"

Installed Packages Microsoft.SemanticKernel, 1.3.0 Microsoft.SemanticKernel.Planners.Handlebars, 1.3.0-preview

In [2]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Planning;


In [3]:
#!import ../Utils/Settings.cs
#!import ../../../Plugins/CustomPlugin/CompanySearchPlugin.cs

In [4]:
using System;
using System.Collections.Generic;
using System.Diagnostics;
using System.IO;
using System.Net.Http;
using System.Text.Json;
using System.Threading.Tasks;

In [5]:
Kernel kernel = Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion("Your Azure OpenAI Service gpt-35-turbo-16k Deployment Name", Settings.AOAIEndpoint, Settings.AOAIKey)
            .Build();

In [6]:
var companySearchPluginObj = new CompanySearchPlugin();
var companySearchPlugin = kernel.ImportPluginFromObject(companySearchPluginObj, "CompanySearchPlugin");

In [7]:
var pluginDirectory = Path.Combine("../../..", "plugins");

In [8]:
var writetPlugin =  kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginDirectory, "WriterPlugin"));
var emailPlugin =  kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginDirectory, "EmailPlugin"));
var translatePlugin =  kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginDirectory, "TranslatePlugin"));

In [9]:
string goal =  """
Check the weather in Guangzhou, use spanish to write emails abount dressing tips based on the results
""";

In [10]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060

var planner = new HandlebarsPlanner();

In [11]:
#pragma warning disable SKEXP0060

var originalPlan = await planner.CreatePlanAsync(kernel, goal);

In [12]:
Console.WriteLine(originalPlan);

{{!-- Step 1: Identify key values --}}
{{set "city" "Guangzhou"}}
{{set "language" "spanish"}}

{{!-- Step 2: Call WeatherSearch helper with the city --}}
{{set "weather" (CompanySearchPlugin-WeatherSearch city)}}

{{!-- Step 3: Call Tips helper with the weather and language --}}
{{set "tips" (WriterPlugin-Tips input=weather language=language)}}

{{!-- Step 4: Call WeatherMail helper with the tips --}}
{{set "email" (EmailPlugin-WeatherMail input=tips)}}

{{!-- Step 5: Print the resulting email --}}
{{json email}}


In [13]:
originalPlan.ToString()

{{!-- Step 1: Identify key values --}}
{{set "city" "Guangzhou"}}
{{set "language" "spanish"}}

{{!-- Step 2: Call WeatherSearch helper with the city --}}
{{set "weather" (CompanySearchPlugin-WeatherSearch city)}}

{{!-- Step 3: Call Tips helper with the weather and language --}}
{{set "tips" (WriterPlugin-Tips input=weather language=language)}}

{{!-- Step 4: Call WeatherMail helper with the tips --}}
{{set "email" (EmailPlugin-WeatherMail input=tips)}}

{{!-- Step 5: Print the resulting email --}}
{{json email}}

In [14]:
#pragma warning disable SKEXP0060
var originalPlanResult = await originalPlan.InvokeAsync(kernel, new KernelArguments());

In [15]:
Console.WriteLine(originalPlanResult);

Queridos colegas de viaje,

Espero que estén teniendo un buen viaje hasta ahora. Quería recordarles que estén atentos a las condiciones de viaje en todo momento, especialmente considerando el clima en Guangzhou. Según los consejos de vestimenta que recibimos, se espera que el clima sea de 2 grados y lluvioso.

Aquí hay algunas recomendaciones para vestirse adecuadamente:

1. Vístete en capas: Debido a las bajas temperaturas, es importante vestirse en capas para mantenerse abrigado. Recuerden llevar una camiseta de manga larga debajo de un suéter o chaqueta.

2. Lleven un abrigo impermeable: Dado que el clima es lluvioso, es recomendable llevar un abrigo impermeable o una chaqueta con capucha para protegernos de la lluvia.

3. Utilicen botas o zapatos impermeables: Para evitar que nuestros pies se mojen, es mejor usar botas o zapatos impermeables durante los días lluviosos.

4. No olviden llevar una bufanda y guantes: Estos accesorios nos ayudarán a protegernos del frío y el viento. Son